In [30]:
# driveのマウント
from google.colab import drive
drive.mount('/content/drive')

# モデルのバージョン
VER = '006'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# ライブラリのインポート

In [31]:
import gc
import os
import re
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

!pip install japanize-matplotlib
import japanize_matplotlib
import folium


from tqdm import tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# データのロード

In [32]:
# パスは適宜変更してください

# ディレクトリを移動
os.chdir('/content/drive/MyDrive/signate-908-hiroshima/')
path = os.getcwd()

# データ読み込み
water_data = pd.read_csv(os.path.join(path, 'train', 'waterlevel', 'data.csv'))
water_stations = pd.read_csv(os.path.join(path, 'train', 'waterlevel', 'stations.csv'))
# rain_data = pd.read_csv(os.path.join(path, 'train', 'rainfall', 'data.csv'))
# rain_stations = pd.read_csv(os.path.join(path, 'train', 'rainfall', 'stations.csv'))
# tide_data = pd.read_csv(os.path.join(path, 'train', 'tidelevel', 'data.csv'))
# tide_stations = pd.read_csv(os.path.join(path, 'train', 'tidelevel', 'stations.csv'))

# データ前処理

In [33]:
def preprocess_water_data_station(water_data, water_stations):
    """water_dataのstationの前処理を行う
    """
    # 欠損値補完
    water_data['river'] = water_data['river'].replace('\u3000', '沼田川')

    # (国)への変更前観測所名を変換
    stations = water_data.loc[water_data['station'].str.contains(r'\(国\)'), 'station'].unique()
    # 中野、伊尾、和木は(国)を含まない観測所が別途存在するため別処理
    stations = [x.replace('(国)', '') for x in stations if x not in ['中野(国)', '伊尾(国)', '和木(国)']]
    water_data['station'] = water_data['station'].apply(lambda x: x + '(国)' if x in stations else x)
    # 中野、伊尾、和木は河川名で分けて処理
    water_data.loc[(water_data['station']=='中野')&(water_data['river']=='太田川'), 'station'] = '中野(国)'
    water_data.loc[(water_data['station']=='伊尾')&(water_data['river']=='芦田川'), 'station'] = '伊尾(国)'
    water_data.loc[(water_data['station']=='和木')&(water_data['river']=='小瀬川'), 'station'] = '和木(国)'

    # (電)への変更前観測所名を変換
    stations = water_data.loc[water_data['station'].str.contains(r'\(電\)'), 'station'].unique()
    stations = [x.replace('(電)', '') for x in stations]
    water_data['station'] = water_data['station'].apply(lambda x: x + '(電)' if x in stations else x)

    # 入力ミスと思われるもの
    water_data['station'] = water_data['station'].replace({'藤波': '藤浪',
                                                           '中州橋': '中洲橋',
                                                           '段原': '段原(猿猴川)'})
    water_data['river'] = water_data['river'].replace({'手越川': '手城川',
                                                       '横川': '横川川'})

    # 入力されない観測所を削除
    in_stations = water_stations.loc[water_stations['入力時使用']==1, '観測所名称'].unique()
    water_data = water_data[water_data['station'].isin(in_stations)]

    return water_data


def preprocess_rain_data_station(rain_data):
    """rain_dataの前処理
    """
    # 重複行を削除
    rain_data = rain_data.drop_duplicates()

    # 2191日分のデータがある観測所のみ使用する
    stations = rain_data.groupby(['station', 'city']).count()['date'].reset_index()
    stations = stations.loc[stations['date']==2191, 'station'].unique()
    rain_data = rain_data[rain_data['station'].isin(stations)]

    return rain_data

In [34]:
# 水位データの観測所をクリーニング
water_data = preprocess_water_data_station(water_data, water_stations)
# rain_data = preprocess_rain_data_station(rain_data)

In [35]:
# 水位データを入力形式に整形
# ほぼrun.pyからコピペ
stations = set(water_stations[water_stations['評価対象']==1]['観測所名称'])

in_all_data = {}

print('processing water data')
for data in tqdm(water_data.groupby('date')):
    day = data[0]
    data_dict = data[1].to_dict('records')
    in_data = []
    for d in data_dict:
        for k in d.keys():
            if k not in ('date', 'station', 'river'):
                in_data.append({'station':d['station'], 'river':d['river'], 'hour':int(k.split(':')[0]), 'value':d[k]})
    in_all_data[day] = {}
    in_all_data[day]['date'] = day
    in_all_data[day]['stations'] = stations
    in_all_data[day]['waterlevel'] = in_data

print('done')
# print('processing rain data')

# for data in tqdm(rain_data.groupby('date')):
#     day = data[0]
#     data_dict = data[1].to_dict('records')
#     in_data = []
#     for d in data_dict:
#         for k in d.keys():
#             if k not in ('date', 'station', 'city'):
#                 in_data.append({'station':d['station'], 'city':d['city'], 'hour':int(k.split(':')[0]), 'value':d[k]})
#     in_all_data[day]['rainfall'] = in_data

# print('done')


processing water data


100%|██████████| 2191/2191 [00:35<00:00, 61.20it/s]

done


In [36]:
# 入力データをdfに変換
water_df = []
# rain_df = []
for d in tqdm(range(len(in_all_data))):
    tmp = pd.DataFrame(in_all_data[d]['waterlevel'])
    tmp['date'] = d
    water_df.append(tmp)

    # tmp = pd.DataFrame(in_all_data[d]['rainfall'])
    # tmp['date'] = d
    # rain_df.append(tmp)

water_df = pd.concat(water_df)
# rain_df = pd.concat(rain_df)

100%|██████████| 2191/2191 [00:11<00:00, 190.16it/s]


In [37]:
# 水位データを数値に変換
water_df['value'] = pd.to_numeric(water_df['value'], errors='coerce')
# rain_df['value'] = pd.to_numeric(rain_df['value'], errors='coerce')
# rain_df.isna().sum()

# 水系名を追加
water_df = water_df.merge(water_stations[['観測所名称', '河川名', '水系名']], left_on=['station', 'river'], right_on=['観測所名称', '河川名'], how='left').drop(['観測所名称', '河川名'], axis=1)

In [38]:
# # 水位観測所のプロット
# # systems = water_stations['水系名'].unique()
# systems = ['江の川', '太田川']

# map = folium.Map(location=['34.5447', '132.8134'], zoom_start=10, tiles='OpenStreetMap')
# water_station = water_stations[water_stations['水系名'].isin(systems)].dropna(subset=['緯度', '経度'])

# for i in range(len(water_station)):
#     folium.Circle(radius=150, location=[water_station.iloc[i]['緯度'], water_station.iloc[i]['経度']],
#                   tooltip=str(water_station.iloc[i]['観測所名称'] + ' (' + water_station.iloc[i]['水系名'] + ')'),
#                   color='red', fill=False).add_to(map)

# map

In [39]:
# # 観測所ごとに1日の平均をプロット
# fig, axes = plt.subplots(90, 2, figsize=(20, 270), tight_layout=True)
# water_df = water_df.sort_values('水系名')
# for i, station in enumerate(tqdm(water_df['station'].unique())):
#     ax = axes[i//2, i%2]
#     df = water_df[water_df['station']==station]
#     river = df.iloc[0]['river']
#     system = df.iloc[0]['水系名']
#     df.groupby('date').mean()['value'].plot(ax=ax)
#     ax.set_xlim(0, 2200)
#     ax.set_title(f'観測所: {station} / 河川: {river} / 水系: {system} / NaNの比率: {round(df["value"].isna().sum()/len(df)*100, 2)}%')
# fig.show()

In [40]:
# 欠損値を線形補完
dfs = []
for group in tqdm(water_df.groupby('station')):
    df = group[1]
    dfs.append(df.interpolate())
water_df = pd.concat(dfs)

# 観測所ごとに1日の平均をプロット
# fig, axes = plt.subplots(90, 2, figsize=(20, 270), tight_layout=True)
# for i, station in enumerate(tqdm(water_df['station'].unique())):
#     ax = axes[i//2, i%2]
#     df = water_df[water_df['station']==station]
#     df.groupby('date').mean()['value'].plot(ax=ax)
#     ax.set_xlim(0, 2200)
#     ax.set_title(f'{station} / NaNの比率: {round(df["value"].isna().sum()/len(df)*100, 2)}%')
# fig.show()

100%|██████████| 178/178 [00:01<00:00, 103.09it/s]


In [41]:
# 不要な変数を削除してメモリを解放
# del water_data, water_stations, rain_data, rain_stations, stations, in_all_data, data_dict, in_data, df, dfs, tmp
# gc.collect()

In [42]:
# ワイドフォーマットに変更
# rain_df = rain_df[['station', 'date', 'hour', 'value']].pivot_table(index=['date', 'hour'], columns='station', values='value').reset_index()

In [43]:
# メモリ削減のためにデータ型を変更
# water_df['value'] = water_df['value'].astype('float32')
# rain_df.iloc[:, 2:] = rain_df.iloc[:, 2:].astype('float32')

In [44]:
# 水位データと降水量データをマージ
# water_df = water_df.merge(rain_df, on=['date', 'hour'], how='left')

# del rain_df
# gc.collect()

# 特徴量エンジニアリング

In [45]:
# 目的変数とシフト特徴量を作る
dfs = []
for group in tqdm(water_df.groupby('station')):
    df = group[1]
    for h in [-24]:
        if h==-24:
            df['y'] = df['value'].shift(h)
        else:
            col = 'shift_' + str(h) + 'h'
            df[col] = df['value'].shift(h)
            col = 'diff_' + str(h) + 'h'
            df[col] = df['value'].diff(periods=h)
    dfs.append(df)
water_df = pd.concat(dfs, axis=0).reset_index(drop=True)

# del df, dfs
# gc.collect()

100%|██████████| 178/178 [00:02<00:00, 78.16it/s] 


In [46]:
value_0h = water_df.loc[water_df['hour']==0, ['date', 'station', 'value']]
value_23h = water_df.loc[water_df['hour']==23, ['date', 'station', 'value']]
water_df = water_df.merge(value_0h, on=['date', 'station'], how='left', suffixes=('', '_0h'))
water_df = water_df.merge(value_23h, on=['date', 'station'], how='left', suffixes=('', '_23h'))
water_df['diff_0tonow'] = water_df['value'] - water_df['value_0h']
water_df['diff_nowto23'] = water_df['value_23h'] - water_df['value']
water_df['diff_0to23'] = water_df['value_23h'] - water_df['value_0h']

# モデリング

In [47]:
# ライブラリインポート
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit

In [48]:
def get_rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [49]:
# ラベルエンコーディング
label_enc_col = ['station', 'river']
for col in label_enc_col:
    le = LabelEncoder()
    water_df[col] = le.fit_transform(water_df[col].values)

In [50]:
water_df.head()

,station,river,hour,value,date,水系名,y,value_0h,value_23h,diff_0tonow,diff_nowto23,diff_0to23
0,0,59,0,1.64,0,沼田川,1.64,1.64,1.64,0.0,0.0,0.0
1,0,59,1,1.64,0,沼田川,1.64,1.64,1.64,0.0,0.0,0.0
2,0,59,2,1.64,0,沼田川,1.64,1.64,1.64,0.0,0.0,0.0
3,0,59,3,1.64,0,沼田川,1.64,1.64,1.64,0.0,0.0,0.0
4,0,59,4,1.64,0,沼田川,1.64,1.64,1.64,0.0,0.0,0.0


In [51]:
# 特徴量カラム
features = ['date', 'hour', 'station', 'river', 'value_23h']

# 欠損値のある行は削除
water_df = water_df[features + ['y']].dropna()

In [52]:
# 学習のパラメータ
params = {
    'objective': 'regression',
    'random_state': 42,
    'boosting_type': 'gbdt',
    'n_estimators': 10000
    }
verbose_eval = 1


# TimeSeriesSplitでCV
ts = TimeSeriesSplit(n_splits=6)

models = []
scores = []


date = water_df['date'].unique()
for date_tr, date_val in ts.split(date):
    train = water_df[water_df['date'].isin(date_tr)]
    valid = water_df[water_df['date'].isin(date_val)]

    X_train, y_train = train[features], train['y']
    X_valid, y_valid = valid[features], valid['y']

    model = lgb.LGBMRegressor(**params)
    model.fit(X_train, y_train, 
            eval_metric='rmse',  # early_stoppingの評価指標(学習用の'metric'パラメータにも同じ指標が自動入力される)
            eval_set=[(X_valid, y_valid)],
            early_stopping_rounds=50,
            verbose=verbose_eval
            )
    
    oof_train = model.predict(X_valid, num_iteration=model.best_iteration_)

    models.append(model)
    scores.append(get_rmse(y_valid, oof_train))

[1]	valid_0's rmse: 25.4929	valid_0's l2: 649.889
Training until validation scores don't improve for 50 rounds.
[2]	valid_0's rmse: 22.9252	valid_0's l2: 525.566
[3]	valid_0's rmse: 20.6151	valid_0's l2: 424.983
[4]	valid_0's rmse: 18.5369	valid_0's l2: 343.617
[5]	valid_0's rmse: 16.6676	valid_0's l2: 277.808
[6]	valid_0's rmse: 14.9863	valid_0's l2: 224.589
[7]	valid_0's rmse: 13.4744	valid_0's l2: 181.56
[8]	valid_0's rmse: 12.1151	valid_0's l2: 146.775
[9]	valid_0's rmse: 10.8933	valid_0's l2: 118.664
[10]	valid_0's rmse: 9.7954	valid_0's l2: 95.9499
[11]	valid_0's rmse: 8.80925	valid_0's l2: 77.6028
[12]	valid_0's rmse: 7.92389	valid_0's l2: 62.788
[13]	valid_0's rmse: 7.12943	valid_0's l2: 50.8288
[14]	valid_0's rmse: 6.41709	valid_0's l2: 41.179
[15]	valid_0's rmse: 5.77898	valid_0's l2: 33.3966
[16]	valid_0's rmse: 5.20795	valid_0's l2: 27.1227
[17]	valid_0's rmse: 4.69769	valid_0's l2: 22.0683
[18]	valid_0's rmse: 4.24244	valid_0's l2: 17.9983
[19]	valid_0's rmse: 3.83716	vali

In [53]:
print(scores)

[1.3179053821930575, 1.0794245916438507, 0.16843268983610354, 4.634657172531895, 0.33931059350684156, 0.18290500468706114]


In [54]:
print('[best_iteration]')
for i, model in enumerate(models):
    print(f'fold {i}: {model.best_iteration_}')
    col = 'importance_' + str(i)
    if i == 0:
        results = pd.DataFrame({
            'feature': features,
            col: model.feature_importances_
            })
    else:
        results[col] = model.feature_importances_
print('[feature_importances]')
display(results)

[best_iteration]
fold 0: 45
fold 1: 300
fold 2: 196
fold 3: 66
fold 4: 176
fold 5: 346
[feature_importances]


,feature,importance_0,importance_1,importance_2,importance_3,importance_4,importance_5
0,date,64,2166,1259,209,868,2076
1,hour,262,1936,1264,376,1281,2253
2,station,320,1506,1060,483,1036,1894
3,river,226,983,600,211,563,1276
4,value_23h,478,2409,1697,701,1532,2881


In [55]:
# 全データで再学習
params = {
    'objective': 'regression',
    'random_state': 42,
    'boosting_type': 'gbdt',
    'n_estimators': int(round(models[-1].best_iteration_*1.2, 0))
    }

model = lgb.LGBMRegressor(**params)
model.fit(water_df[features], water_df['y'])

LGBMRegressor(n_estimators=415, objective='regression', random_state=42)

# モデル保存

In [56]:
import pickle

In [57]:
model_file = 'baseline_lightgbm_' + VER + '.pkl'
model_path = os.path.join(path, 'models', model_file)
with open(model_path, 'wb') as f:
    pickle.dump(model, f)